In [1]:
!pip install torch torchvision pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 2.4 MB/s eta 0:00:0000:0100:01m


In [2]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from pycocotools.coco import COCO
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [3]:
class DhakaTrafficDataset(Dataset):
    def __init__(self, root, annotation, transform=None):
        self.root = root
        self.coco = COCO(annotation)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        img_metadata = coco.loadImgs(img_id)[0]
        path = os.path.join(self.root, img_metadata['file_name'])

        image = Image.open(path).convert("RGB")
        boxes = []
        labels = []

        for ann in anns:
            xmin = ann['bbox'][0]
            ymin = ann['bbox'][1]
            xmax = xmin + ann['bbox'][2]
            ymax = ymin + ann['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(ann['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        if self.transform:
            image = self.transform(image)

        return image, target

# Define transforms with data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create datasets
train_dataset = DhakaTrafficDataset(
    root="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/train",
    annotation="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/train/_combined_annotations.coco.json",
    transform=transform
)

val_dataset = DhakaTrafficDataset(
    root="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/valid",
    annotation="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/valid/_combined_annotations.coco.json",
    transform=transform
)

test_dataset = DhakaTrafficDataset(
    root="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/test",
    annotation="/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/test/_combined_annotations.coco.json",
    transform=transform
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


loading annotations into memory...
Done (t=1.31s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.39s)
creating index...
index created!


In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the head of the model for the number of classes in the dataset
num_classes = 10  # 9 object classes + 1 background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move model to the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 158MB/s]  


In [11]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch} Average Loss: {avg_loss}")

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)

            # Ensure loss_dict is a dictionary
            if isinstance(loss_dict, dict):
                losses = sum(loss for loss in loss_dict.values())
                total_loss += losses.item()
            else:
                # Handle case where loss_dict is not as expected
                print("Unexpected loss_dict format:", loss_dict)
    
    avg_loss = total_loss / len(data_loader)
    return avg_loss


In [ ]:
# Function to verify data loader
def verify_data_loader(data_loader, coco_annotation_file):
    # Load COCO annotations from the provided file
    with open(coco_annotation_file, 'r') as f:
        coco_data = json.load(f)
    
    # Get annotations for the first image from the JSON file
    image_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == 0]
    
    for images, targets in data_loader:
        # Extract the first image and its annotations
        image = images[0]
        target = targets[0]
        
        # Print the shape of the image and the targets
        print("Image Shape:", image.shape)
        print("Targets:", target)
        
        # Compare the targets with annotations from the JSON file
        for i, bbox in enumerate(target['boxes']):
            if i < len(image_annotations):
                coco_bbox = image_annotations[i]['bbox']
                coco_label = image_annotations[i]['category_id']
                
                print("Predicted BBox:", bbox)
                print("COCO BBox:", coco_bbox)
                print("Predicted Label:", target['labels'][i])
                print("COCO Label:", coco_label)
                print()
            else:
                print("No more annotations in COCO data")
                break
        break

# Verify the data loader for the first image
verify_data_loader(train_loader, "/kaggle/input/dhaka-city-traffic-2024/Final_Dhaka_Traffic_Dataset/train/_combined_annotations.coco.json")


In [12]:
num_epochs = 20
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
#     val_loss = evaluate(model, val_loader, device)
#     print(f"Epoch {epoch} Validation Loss: {val_loss}")
    lr_scheduler.step()
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')

# Save the final model
torch.save(model.state_dict(), 'faster_rcnn_dhaka_traffic_final.pth')


Epoch 0 Average Loss: 0.5877125271998639
Epoch 1 Average Loss: 0.5449087815969271
Epoch 2 Average Loss: 0.5063790909002441
Epoch 3 Average Loss: 0.4060862099662358
Epoch 4 Average Loss: 0.3733894415362433
Epoch 5 Average Loss: 0.3500240932596953
Epoch 6 Average Loss: 0.3246603087322811
Epoch 7 Average Loss: 0.3210463280256789
Epoch 8 Average Loss: 0.3180305777711379
Epoch 9 Average Loss: 0.31519344845160385
Epoch 10 Average Loss: 0.3145954629495805
Epoch 11 Average Loss: 0.31449836255844243
Epoch 12 Average Loss: 0.3138036439642635
Epoch 13 Average Loss: 0.313688265851101
Epoch 14 Average Loss: 0.313716301393561
Epoch 15 Average Loss: 0.31372223408917155
Epoch 16 Average Loss: 0.31361268756008304
Epoch 17 Average Loss: 0.31347067068497686
Epoch 18 Average Loss: 0.3138335367428814
Epoch 19 Average Loss: 0.3140278331056953


In [ ]:
# Load the saved model
model_path = 'faster_rcnn_dhaka_traffic_final.pth'
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 10  # Adjust based on your dataset
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load(model_path))
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Continue training for additional epochs
num_epochs_additional = 10
for epoch in range(num_epochs_additional):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()
    torch.save(model.state_dict(), f'model_epoch_{epoch + num_epochs}.pth')

# Save the final model
torch.save(model.state_dict(), 'faster_rcnn_dhaka_traffic_final_updated.pth')
